In [58]:
import glob, re, os, shutil, random
import pandas as pd
from pathlib import Path
import numpy as np

In [9]:
x = glob.glob("data/align_image_1_3/*/*")
for filename in x:
    fre = re.search('data\/align_image_1_3\/([0-9]*)\/([0-9]*).jpg',filename, re.IGNORECASE)
    if fre:
        folder = fre.group(1)
        file = fre.group(2)
        new_name = f"data/renamed/{folder}_{file}.jpg"
        shutil.copy(filename, new_name)

In [96]:
#############################################
######### Split label file E/I, ... #########
#############################################

TARGET_LIST = ['E/I', 'F/T', 'J/P', 'N/S']
for TARGET in TARGET_LIST:
    df = pd.read_csv("data/label.csv", index_col= "No.")
    df.drop(df.columns[-3:], inplace=True, axis = 1)
    df.rename(columns={df.columns[0]:'MBTI'}, inplace=True)
    EI_df = pd.get_dummies(df[TARGET])
    EI_df.insert(0, "filename", 0)
    temp = EI_df.loc[EI_df.index.repeat(5)]
    temp.index = np.arange(1, len(temp) + 1)
    for index in range(len(temp.index)):
            filename = f"{index//5+1}_{index%5+1}.jpg"
            temp.iloc[index, temp.columns.get_loc("filename")] = filename

    temp.rename(columns=lambda x: x.strip(), inplace=True)
    temp.to_csv(f"data/label_{TARGET[0]}_{TARGET[2]}.csv", index = False)


In [62]:
#############################################
##### Select Validation data file, ... ######
#############################################

###############
# Don't run!
###############

NUM = 394
TRAIN_SIZE = int(394 * 0.8)
VAL_SIZE = NUM-TRAIN_SIZE
IMG_LIST = [x+1 for x in range(NUM)]
VAL_LIST = random.sample(IMG_LIST, VAL_SIZE)

In [121]:
###################################
##### Train, Validation Split #####
###################################

#########################
CLASS = ('E', 'I')
#########################


# Create missing directory
for dir in ["train", "validation"]:
    for c in CLASS:
        Path(f"data/EI/{dir}/{CLASS[0]}").mkdir(parents=True, exist_ok=True)
        Path(f"data/EI/{dir}/{CLASS[1]}").mkdir(parents=True, exist_ok=True)
df = pd.read_csv(f"data/label_{CLASS[0]}_{CLASS[1]}.csv" )

# lambda function for PATH setting
TARGET_PATH = lambda target, cl : f"data/{CLASS[0]}{CLASS[1]}/{target}/{cl}"
SOURCE_PATH = lambda name : f"data/renamed/{name}"

# iterate over index and copy data from source path
for index, data in df.iterrows():
    t = "validation" if index//5 +1 in VAL_LIST else "train"
    if data.loc[CLASS[0]]==1:
        target_class = CLASS[0]
    else:
        target_class = CLASS[1]
    try:
        shutil.copy(SOURCE_PATH(data["filename"]), TARGET_PATH(t, target_class))
    except:
        pass


